In [1]:
import numpy as np                                                    # Packages for data analysis
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
from scipy.ndimage import gaussian_filter1d
import braingeneers                                                   # Braingeneers code
from braingeneers.analysis.analysis import SpikeData, read_phy_files
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets 
import ipywidgets as ipw
from IPython.display import HTML, display, Javascript, clear_output
from scipy.sparse import csr_matrix
import scipy.io
from scipy.sparse import coo_matrix
import numpy as np
from decimal import Decimal, getcontext

In [2]:
data = scipy.io.loadmat('Figure_4.mat')
fig_4_spikes = data['spks']
example_data = np.array(fig_4_spikes)

In [3]:
example_data[:,0]

array([ 15.,  15.,  15., ..., 541., 541., 541.])

In [4]:
def createFCM(spksExc, spksInh=None):    
    
    # time constant of exponential profile (used for coupling)
    tau = 3e-3

    # 1e-4 binless window step in seconds
    step = 1e-4
    
    # store ids of neurons that really fired -- corresponding to entries in
    # adjacency relation
    
    if spksInh is None:
        old_ids = np.unique(spksExc[:, 0])
        spks = np.column_stack((spksExc, np.ones(len(spksExc))))
    else:
        old_ids = np.unique(np.concatenate((spksExc[:, 0], spksInh[:, 0])))
        spks = np.row_stack((np.column_stack((spksExc, np.ones(len(spksExc)))), 
                             np.column_stack((spksInh, -np.ones(len(spksInh))))))
        
    Eneuron_end = np.max(spksExc[:, 0])
    num_neurons = len(old_ids)
    
    row_indices = old_ids - 1  # Since Python uses 0-based indexing, we subtract 1 from old_ids
    col_indices = np.zeros_like(row_indices)
    values = np.arange(1, num_neurons + 1)

    new_ids = coo_matrix((values, (row_indices, col_indices)), shape=(num_neurons, 1))
    
    # frequency of spiking for individual neurons
    spike_freq = np.zeros(num_neurons)
    
    # duration of spike trains
    t_min = 0
    t_max = np.max(spksExc[:,1])
    T = t_max - t_min
    sig_length = np.ceil(T/step).astype(int)
    
    # Convert the new_ids sparse matrix to a dense NumPy array
    new_ids_array = new_ids.toarray().flatten()

    print(Eneuron_end)

    # Get spiking frequencies
    spike_freq = np.zeros(num_neurons)

    for i in range(1, num_neurons + 1):
        if i <= Eneuron_end:
            neuron_spikes = spksExc[spksExc[:, 0] == old_ids[i - 1], 1]
            spike_freq[i - 1] = np.sum(neuron_spikes) / T
        else:
            if spksInh is None:
                print("spksInh is None. Make sure it is properly defined.")
                return None
                
            neuron_spikes = spksInh[spksInh[:, 0] == old_ids[i - 1], 1]
            spike_freq[i - 1] = np.sum(neuron_spikes) / T
            
    # create signals for each neuron        
    signals = np.zeros((num_neurons, sig_length))
    for i in range(1, num_neurons + 1):
        k = old_ids[i - 1]
        IorE = np.unique(spks[spks[:, 0] == k, 2])
        # how neuron i will influence other neurons
        signals[i - 1, :] = create_signal(spks[spks[:, 0] == k, 1], step, sig_length, tau, IorE)
    
    FC_MAT = np.zeros((num_neurons, num_neurons))
    spike_vecs = np.zeros((num_neurons, sig_length))

    # combine with discrete spiking events
    for i in range(1, num_neurons + 1):
        k = old_ids[i - 1]
        spike_times = spks[spks[:, 0] == k, 1]
        spike_times_vec = coo_matrix((np.ones_like(spike_times), 
                                      (np.ceil(spike_times / step).astype(int) - 1, 
                                       np.zeros_like(spike_times))), 
                                     shape=(sig_length, 1)).toarray().flatten()
        spike_vecs[i - 1, :] = spike_times_vec
        # influence on neuron i from all other neurons.
        FC_MAT[:, i - 1] = signals.dot(spike_times_vec)

    return FC_MAT

In [5]:
def create_signal(start_times, dt, sig_length, tau, mode):
    # time vector
    t_vec = np.arange(dt, (sig_length + 1) * dt, dt)

    q = np.zeros_like(t_vec)

    if mode == 1:
        for k in range(len(start_times)):
            qq = np.exp(-(t_vec - start_times[k]) / tau)
            qq[qq > 1] = 0
            if k < len(start_times) - 1:
                qq[t_vec >= start_times[k + 1]] = 0
            q = q + qq
    else:
        for k in range(len(start_times)):
            qq = np.exp(-(t_vec - start_times[k]) / tau)
            qq[qq > 1] = 0
            qq = 1 - qq
            qq[qq > 0.99] = 0  # corresponds to ~4.5 tau
            if k < len(start_times) - 1:
                qq[t_vec >= start_times[k + 1]] = 0
            q = q + qq

    # due to rounding of spiking times, the exponential may sometimes not start at 1
    if mode == 1:
        idx = np.ceil(start_times / dt).astype(int) - 1
        idx = np.clip(idx, 0, len(q) - 1)  # Clip indices to be within valid range
        q[idx] = 1

    # shift to zero mean and make magnitude normalized
    q = q - np.mean(q)
    q = q / (np.max(q) * len(start_times))

    return q

In [6]:
FC_MAT = createFCM(example_data)

800.0


/tmp/ipykernel_935048/3072996499.py:9: RuntimeWarning: overflow encountered in exp
  qq = np.exp(-(t_vec - start_times[k]) / tau)


In [8]:
FC_MAT

array([[ 1.00000000e+00, -3.20770399e-02, -6.91855246e-03, ...,
         3.31705909e-02, -1.30781348e-02, -4.22800199e-03],
       [-2.57603388e-02,  1.00000000e+00, -1.38334218e-02, ...,
        -1.62846502e-02,  6.26150717e-02, -2.15501397e-02],
       [ 2.33915558e-02, -2.43913911e-02,  1.00000000e+00, ...,
         6.55725244e-02,  5.76372692e-04,  2.54416000e-02],
       ...,
       [-6.71858639e-04, -8.98475320e-03,  7.59342292e-02, ...,
         1.00000000e+00, -1.66043219e-02,  9.10637226e-02],
       [-7.28024613e-03,  7.16004208e-02, -1.04725694e-02, ...,
        -2.76453515e-02,  1.00000000e+00, -2.79230507e-02],
       [-9.60234497e-04,  1.06505340e-02,  1.10421402e-01, ...,
         8.26998989e-02, -5.90451886e-03,  1.00000000e+00]])

In [15]:
signals[0,15]

In [11]:
spike_freq

array([20.94499224, 24.32624255, 26.02272386, 23.57765654, 20.45808099,
       22.42074178, 22.38054958, 20.43272937, 22.40372391, 20.94133841,
       26.00528054, 23.5905701 , 22.3922869 , 24.31433005, 20.45255018,
       26.7663797 , 23.61739827, 26.72093198, 24.29593573, 20.92359477,
       24.29320787, 20.45177436, 20.4044747 , 20.41255819, 26.02607738,
       24.31815907, 26.76785625, 24.32549177, 26.04276991, 24.31648231,
       24.28512438, 26.01992092, 26.03265929, 22.39601582, 24.31843436,
       22.38375294, 24.3041944 , 24.27398769, 26.73189349, 26.68814755,
       26.6935282 , 22.40732769, 20.95707993, 22.38074979, 26.73422093,
       26.71920517, 26.76112418, 25.9823815 , 26.69515491, 20.4245708 ,
       20.49309275, 24.30386906, 24.27764152, 20.46716552, 26.0280044 ,
       20.92021623, 20.42714851, 23.59182141, 26.75143901, 20.42890035,
       23.60108114, 24.34568797, 20.42186796, 26.03541218, 24.30284299,
       26.73049202, 26.73447119, 26.75947245, 22.41831423, 23.63

In [10]:
spike_freq

In [7]:
np.zeros(8)

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
np.max(example_data[:, 0])

800.0

In [17]:
len(ids)

800